# First pass at image processing

1. Motion correction (FIJI)
2. Bleach correction (FIJI)
3. Load data into python, load ryan's sync pulse CSV to get the stim timing
4. for each stim pulse, average the stim pulse frames
5. for each stim pulse, collect a few pre-stim samples and average those
6. for each stim pulse, subtract pre-stim average from stim average
7. Average the subtracted images across however many trials we did

## Load image

In [ ]:
import skimage as ski
import pandas as pd
import numpy as np

spamming_exp = False # True for spamming experiment, False for 5s On/10s off experiment
if spamming_exp:
    img = ski.io.imread('/Users/ilya_arcadia/Neuroimaging/Data/DUP_right_blue_spamming_1_MMStack_Default_aligned_bleachcorr.ome.tif') # ignore_frames_beginning = 200
    ignore_frames_beginning = 200 # ignore first N frames (due to motion etc)
else:
    img = ski.io.imread('/Users/ilya_arcadia/Neuroimaging/Data/right_blue_1_MMStack_Default_aligned_bleachcorr.ome.tif') # ignore_frames_beginning = 0
    ignore_frames_beginning = 0 # ignore first N frames (due to motion etc)

img = img[ignore_frames_beginning:]

## Load sync pulse data from DAQ

In [ ]:
min_stim_length = 3 # minimum length of pinch (in frames)

if spamming_exp:
    sync_file = pd.read_csv('/Users/ilya_arcadia/Neuroimaging/Data/right_blue_spamming_processed.txt', index_col=0)
else:
    sync_file = pd.read_csv('/Users/ilya_arcadia/Neuroimaging/Data/right_blue_processed.txt', index_col=0)

sync_file['pinch'] = sync_file['pinch'].astype(int) # convert True to 1 and False to 0

# find indices of sync pulse onset
onset_idx = sync_file.index[(np.diff(sync_file['pinch'], prepend=0) > 0)]
offset_idx = sync_file.index[(np.diff(sync_file['pinch'], prepend=0) < 0)]

stim_frames_start, stim_frames_end = [], []
for on, off in zip(onset_idx, offset_idx):
    if off - on > min_stim_length and on > ignore_frames_beginning:
        stim_frames_start.append(on-ignore_frames_beginning)
        stim_frames_end.append(off-ignore_frames_beginning)
print('pinch start: ', end='')
print(stim_frames_start)
print('pinch end: ', end='')
print(stim_frames_end)

## Process image

In [ ]:
n_bg_frames = 3 if spamming_exp else 100
max_intensity_diff = 1000 # ignore pixels with > N intensity difference because these are motion artifacts
imgs_subtracted = []
for s,e in zip(stim_frames_start, stim_frames_end):
    img_stim = img[s:e]
    img_bg = img[s-1-n_bg_frames:s-1]
    img_stim_avg = img_stim.mean(axis=0)
    img_bg_avg = img_bg.mean(axis=0)
    img_stim_avg_sub = img_stim_avg - img_bg_avg
    # remove outlier pixels
    img_stim_avg_sub[img_stim_avg_sub > max_intensity_diff] = 0
    img_stim_avg_sub[img_stim_avg_sub < -max_intensity_diff] = 0
    img_stim_avg_sub = img_stim_avg_sub - np.median(img_stim_avg_sub)
    imgs_subtracted.append(img_stim_avg_sub)

imgs_subtracted = np.array(imgs_subtracted)

ski.io.imshow(imgs_subtracted.mean(axis=0), cmap='gray', vmin=-50, vmax=50)
